In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.applications import ResNet50

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224

# Re-loads the MobileNet model without the top or FC layers
ResNet50 = ResNet50(weights = 'imagenet',
                 include_top = False,
                 input_shape = (img_rows, img_cols, 3))


# # Layers are set to trainable as True by default
# for layer in Xception.layers:
#     layer.trainable = True

# # Let's print our layers
# for (i,layer) in enumerate(Xception.layers):
#     print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def addTopModelXception(bottom_model, num_classes):
    """creates the top or head of the model that will be
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 2

FC_Head = addTopModelXception(ResNet50, num_classes)

model = Model(inputs = ResNet50.input, outputs = FC_Head)

print(model.summary())

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/kaggle/input/dogs-vs-cats/dataset/train'
validation_data_dir = '/kaggle/input/dogs-vs-cats/dataset/validation'

# Let's use some data augmentaiton
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size =16

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.optimizers.legacy import RMSprop

# We use a very small learning rate
# model.compile(loss = 'categorical_crossentropy',
#               optimizer = 'adam',
#               metrics = ['accuracy'])

from tensorflow import keras
from keras import optimizers
# optimizer=keras.optimizers.RMSprop(learning_rate=0.01)

# We use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.RMSprop(learning_rate=0.01),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 20000
nb_validation_samples = 5000

# We only train 5 EPOCHS
epochs = 5
batch_size = 16

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
model.save("cat_vs_dog.keras")

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# We need to recreate our validation generator with shuffle = false
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

nb_train_samples = 20000
nb_validation_samples = 5000

#Confution Matrix and Classification Report
Y_pred = model.predict(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

In [ ]:
# epoch=5
import numpy as np
import cv2
from keras.preprocessing import image
from keras.models import load_model
model= load_model('cat_vs_dog.keras')
def predict_one_image(img, model):
  img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
  img = np.reshape(img, (1, 224, 224, 3))
  img = img/255.
  pred = model.predict(img)
  class_num = np.argmax(pred)
  return class_num, np.max(pred)

In [ ]:
# epoch =5
import cv2
import os
from matplotlib import pyplot as plt
# datas=os.listdir("/kaggle/input/dogs-vs-cats/dataset/test")
# test_img = cv2.imread('/kaggle/input/dogs-vs-cats/dataset/test/cats/cat (4752).jpg')
test_img = cv2.imread('/kaggle/input/dogs-vs-cats/dataset/test/cats/cat (5824).jpg')
# test_img = cv2.imread('/kaggle/input/dogs-vs-cats/dataset/test/dogs/dog (211).jpg')

pred, probability = predict_one_image(test_img,model)
print(probability)

if probability > 0.7:
    print('%s %d%%' % (datas[pred], round(probability, 2) * 100))
    _, ax = plt.subplots(1)
    plt.imshow(test_img)
    # Turn off tick labels
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.grid('off')
    plt.show()
else:
    print("This is not a dog" )
#     print('%s %d%%' % (datas[pred], round(probability, 2) * 100))
    plt.imshow(test_img)


# result, probability = predict_one_image(test_img,model)
# print(probability)
# print(result)

# if result == 1:
#   print("This is cat")
# else:
#   print("this is dog")
#   # reading the image
# # testImage = img.imread(image_path)

# # displaying the modified image
# plt.imshow(test_img)
